In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [6]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [8]:
len(df_train), len(df_val)

(3009173, 2855951)

In [9]:
duration_std = df_train['duration'].std() 

In [10]:
duration_std

9.939385620151036

In [11]:
# # Total number of records before removing outliers
# initial_count = len(df_train)

# # Remove outliers by keeping only rows where 'duration' is between 1 and 60 minutes
# df_train_filtered = df_train[(df_train['duration'] >= 1) & (df_train['duration'] <= 60)]

# # Total number of records after removing outliers
# final_count = len(df_train_filtered)

# # Calculate the fraction of records left
# fraction_left = final_count / initial_count

# print(f"Fraction of records left: {fraction_left:.2f}")

In [12]:
# # Cast the IDs to strings
# df_train['PULocationID'] = df_train['PULocationID'].astype(str)
# df_train['DOLocationID'] = df_train['DOLocationID'].astype(str)

# # Turn the DataFrame into a list of dictionaries using only 'PULocationID' and 'DOLocationID'
# data_dicts = df_train[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# # Fit a DictVectorizer
# vectorizer = DictVectorizer(sparse=False)
# feature_matrix = vectorizer.fit_transform(data_dicts)

# # Get the dimensionality of the matrix
# num_columns = feature_matrix.shape[1]

# print(f"The dimensionality of the matrix is: {num_columns} columns")

In [13]:
df_train.keys()

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [14]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

: 

In [15]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715209663881

In [ ]:
import os
import pickle

# Create the 'models' directory if it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')

# Save the model to a file
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)


In [ ]:
import pickle

# Load the model from the file
with open('models/lin_reg.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)


In [ ]:
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715209663881